# Module 3: Exploratory Data Analysis

## Configuration

In [ ]:
# basic configuration, put these lines at the top of each notebook
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# plotting configuration (basically just change plot size)
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10, 6)

In [ ]:
# show all columns of our data frames
import pandas as pd
pd.options.display.max_columns = None
pd.set_option("display.precision", 2)
pd.options.display.max_rows = 100

## Data loading

We have two distinct datasets which we can use for our fraud detection use case:
- Transaction data: this includes information like the target variable (fraud/no fraud) or credit card information
- Identity data: this includes information like device and operating system of the buyer

We can join both datasets using the variable `TransactionID`. More detailed information can be found on the official [competition page](https://www.kaggle.com/c/ieee-fraud-detection/data).

Firstly, let's create a symbolic link to our shared data directory, as well as a directory for storing intermediate results.

In [ ]:
!pwd

In [ ]:
!ln -s ../../shared_data/fraud-detection data

In [ ]:
!ls data

We can see that there are two data files inside our shared data directory. Since it is read-only, we need to create a directory for storing intermediate results.

In [ ]:
!mkdir tmp

In [ ]:
!ls

Now, let's load this data into data frames, i.e., indexed tables. We can use the `pandas` package for this.

In [ ]:
import pandas as pd

In [ ]:
# it's a good practice to use uppercase names for constants
DATA_PATH = 'data/'
TMP_PATH = 'tmp/'
identity = pd.read_csv(f'{DATA_PATH}train_identity.csv')
transaction = pd.read_csv(f'{DATA_PATH}train_transaction.csv')

## Dataset overview

We can now take a look at the number of columns (i.e., features) and rows (i.e., examples) in both datasets.

In [ ]:
print(f'Identity dataset has {identity.shape[0]} rows and {identity.shape[1]} columns.')
print(f'Transaction dataset has {transaction.shape[0]} rows and {transaction.shape[1]} columns.')

Now, we can combine the two datasets using the `merge` function. Since we are primarily interested in transactions, this will be our main table. Thus, we will simply add identity information to each transaction if available. Conceptually, this is equivalent to performing a _left join_ in SQL.

In [ ]:
data = pd.merge(transaction, identity, on='TransactionID', how='left')

print(f'Training dataset has {data.shape[0]} rows and {data.shape[1]} columns.')

In order to get a first impression of our dataset, we can use `pandas`' `head` function, which prints out the first `n` rows.

In [ ]:
data.head(n=10)

For our subsequent steps, it's useful to know the nature of our features. With a tabular dataset like this, we mainly distinguish between *categorical* and *numerical* features. Luckily, Kaggle provides us with more information about how to treat each feature (see image and [forum post](https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203#latest-607486))

<img alt="Categorical features of fraud detection dataset" src="img/kaggle_feature_information.png" style="width: 800px"/>

Let's mark all these features as categorical to derive better descriptive statistics later on. Right now, they are marked as numbers or don't have a meaningful type at all.

In [ ]:
print(f'Type of target variable: {data.isFraud.dtype}')
print(f'Type of feature ProductCD: {data.ProductCD.dtype}')

In [ ]:
cat_feats = ['isFraud', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 
             'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 
             'DeviceType', 'DeviceInfo', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19',
             'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
             'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']

In [ ]:
for feat in cat_feats:
    data[feat] = data[feat].astype('category')

In [ ]:
print(f'Type of target variable: {data.isFraud.dtype}')
print(f'Type of feature ProductCD: {data.ProductCD.dtype}')

Let's quickly check if the data remained unchanged.

In [ ]:
data.head()

Now, we can take a look at the ratio of numerical and categorical features. We do this by looking at the `dtype` attribute of each column. After collecting these, we can use the `Counter` class from the standard library to derive exact counts.

In [ ]:
from collections import Counter

dtypes = [data[feat].dtype.name for feat in data.columns]
c = Counter(dtypes)
type_counts = c.most_common()

for count in type_counts:
    print(f'Dataset contains {count[1]} features of type {count[0]}.')

## Target variable

For our target variable (as for every categorical variable), we can look at the following aspects to get a better understanding of it:
- Number of classes
- Descriptive statistics: count & frequencies
- Plots: histogram

Let's look at all these things...

In [ ]:
print(f'Target variable {data.isFraud.name} has {len(data.isFraud.dtype.categories)} classes.')

In [ ]:
data.isFraud.dtype.categories

In [ ]:
data.isFraud.describe()

In [ ]:
data.isFraud.value_counts(normalize=True)

Our dataset seems to be very imbalanced, i.e., one class is overrepresented. In our case, more than 96% of all transactions were not registered as fraudulent.

Alternatively to the above generated statistics, we can look at a histogram to get a first impression of our target variable.

In [ ]:
data.isFraud.value_counts(normalize=True).plot(kind='bar')

## Features

For our features we can look at univariate statistics, which describe characteristics of one particular feature, and bivariate statistics, which focus on relationships between features. We will start with the former part of the analysis.

### Missing values

Firstly, we should look at how much missing data we have. The `isnull` function counts missing values, as well as `NaN` values.

In [ ]:
missing_vals = data.isnull().sum().sort_values(ascending=False) / len(data)
missing_vals

We can already see that we should exclude a large percentage of our features, probably everything with more than 20% of missing values (since we have a lot of data). Additionally, we should check whether these numbers are similar for both classes. Let's put this functionality inside a function.

In [ ]:
def missing_val_perc(df, levels, target=None):
    data = df
    if target is not None:
        data = data.loc[data.isFraud == target]
    missing_vals = data.isnull().sum().sort_values(ascending=False) / len(data)
    for l in levels:
        perc = len(missing_vals.loc[missing_vals > l]) / len(missing_vals)
        print('Percentage of features with more than {:.0f}% missing values: {:.1f}%'.format(l * 100, perc * 100))

In [ ]:
levels = [0.2, 0.5, 0.8]

missing_val_perc(data, levels, target=0)

In [ ]:
missing_val_perc(data, levels, target=1)

The percentages are roughly equal, excluding columns should thus be no problem.

### Univariate statistics & plots

We already saw an example of how to apply the `describe` function. We can use this to get an impression of our features as well. Let's look at examples for both categorical and numerical variables.

#### Categorical variables

In [ ]:
data.card4.describe()

In [ ]:
data.card4.value_counts(normalize=True)

In [ ]:
data.card4.value_counts(normalize=True).plot(kind='bar')

Histograms for some more interesting (and interpretable) features are shown below. Because some of the categorical variables contain a lot of classes, we will write a helper function that plots only the top ten items.

In [ ]:
def plot_top_ten(col):
    counts = col.value_counts(normalize=True)
    top_ten = counts.head(10)
    top_ten.plot(kind='bar', title=col.name)
    plt.show()

**Exercise:** Describe the variables `card6` and `P_emaildomain` using descriptive statistics and visualizations.

#### Numerical variables

The `describe` function also works for numerical variables. It will offer us basic statistics like the mean, standard deviation and quantiles of a particular column. Let's look at the purchase amount first, which is stored in the `TransactionAmt` column.

In [ ]:
data.TransactionAmt.describe()

In [ ]:
def plot_bins(col, bins, labels):
    bins = pd.cut(col, bins, labels=labels).value_counts(normalize=True)
    bins.plot(kind='bar', title=col.name)
    plt.show()

In [ ]:
bins = [0, 10, 50, 100, 500, 1000, 5000, 10000, 50000]
labels = ['0-9', '10-49', '50-99', '100-499', '500-999', '1,000-4,999', '5,000-9,999', '10,000-50,000']

plot_bins(data.TransactionAmt, bins, labels)

We could repeat this process for the other numerical variables (and you **should** definitely do that for your own data!), but since this is a Kaggle competition most of the features are anonymized and thus not very interpretable.

**Exercise:** Describe the variables `dist1` and `dist2` using descriptive statistics and visualizations.

### Bivariate statistics & plots

After looking at some univariate statistics, we can now go on to examine bivariate relationships within our data. Here, we will mainly look at correlation statistics between features (and the target variable) and create some plots that might give us intuitions about relationships within our dataset.

Since we have two variable types (categorical & numerical), we have to account for three relationship types:
- numerical-numerical
- categorical-categorical
- categorical-numerical

Let's start with the first one.

#### Two numerical variables

Let's look at `TransactionAmt` and `TransactionDT`, since we can interpret these features.

In [ ]:
df = data[['TransactionAmt', 'TransactionDT']].dropna()
df.shape

We can subtract the minimum from all datetimes to get more meaningful data.

In [ ]:
df['TransactionDT'] = df['TransactionDT'] - df['TransactionDT'].min()
df.head(n=10)

Now, we can calculate the correlation coefficient and create some plots.

In [ ]:
df.corr()

In [ ]:
df_sample = df.sort_values('TransactionDT').sample(1000)

In [ ]:
plt.plot(df_sample.TransactionDT, df_sample.TransactionAmt, 'b.')
plt.show()

As expected, there is no pattern to exploit for the transaction date, as indicated by the correlation coefficient (0.01) and visual inspection of the plot.

**Exercise:** Examine the relationship between the variables `TransactionAmt` and `dist1`/`dist2` using the outlined process.

#### Two categorical variables

For two categorical variables, we can create matrices based on value counts and stacked bar charts. As an example, let's look at the relationship between credit card company and fraudulence.

In [ ]:
def plot_fraud_percentage_per_class(data, col, return_df=False):
    # extract and aggregate necessary data
    counts = data[[col, 'isFraud']].dropna().groupby([col, 'isFraud']).size()
    index = []
    percs = []
    # calculate fraud/no fraud percentages and save in data frame
    for level in counts.index.levels[0]:
        level_counts = counts[level]
        no_fraud = level_counts.iloc[0] / level_counts.sum()
        fraud = 0
        if len(level_counts) > 1:
            fraud = level_counts.iloc[1] / level_counts.sum()
        percs.append((fraud, no_fraud))
        index.append(level)
    df = pd.DataFrame(columns=['fraud', 'no_fraud'], index=index, data=percs)
    plt.bar(df.index, df.fraud, width=0.4)
    plt.xlabel(col)
    plt.ylabel('fraud percentage')
    plt.show()
    if return_df:
        return df

In [ ]:
df = plot_fraud_percentage_per_class(data, 'card4', return_df=True)
df

We can see that fraudulent activity is far more common for credit card company "Discover". This could be a valuable insight in model building. Let's plot this for some more variables.

**Exercise:** Examine fraud percentages for the variables `card6` and `DeviceType`.

#### One numerical, one categorical variable

If we are looking at one numerical and one categorical variable, we can for example examine descriptive statistics per class. Let's write a function that aggregates numerical data and creates a DataFrame containing the mean, median and standard deviation for each class.

In [ ]:
def plot_mean_per_class(data, cat, num, return_df=False):
    df = data[[cat, num]].groupby(cat).agg(['mean', 'std', 'median'])
    df[(num, 'mean')].plot(kind='bar')
    plt.show()
    if return_df:
        return df

In [ ]:
df = plot_mean_per_class(data, 'isFraud', 'TransactionAmt', return_df=True)

In [ ]:
df

We can now reuse this function to generate some more plots.

**Exercise:** Examine the relationships of the following variable pairs:
- `DeviceType` and `TransactionAmt`
- `card4` and `TransactionAmt`
- `card6` and `TransactionAmt`

## Save progress

To conclude this part of the ML workflow, let's save our data to a CSV file for later use. We will also look at the file size in order to compare it to the following steps in our workflow.

In [ ]:
data.to_csv(f'{TMP_PATH}data_raw.csv')

In [ ]:
!ls -lh tmp/